## Plots for STRise conference

In [1]:
import logging
logger = logging.getLogger(__name__)
import pypsa
import pandas as pd
import os
import yaml
project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
print(f"Project directory: {project_dir}")

Project directory: /home/bobby/projects/pci-pmi-2030-targets


In [2]:
## append ../scripts to path
import sys
sys.path.append(os.path.join(project_dir, "workflow/scripts"))

In [3]:
configfile = configfile = str(project_dir) + "/config/config.yaml"
config = yaml.safe_load(open(configfile))

In [4]:
configfile_project = str(project_dir) + "/config/config.pci-pmi.yaml"
config_project = yaml.safe_load(open(configfile_project))

prefix = config_project["run"]["prefix"]
scenarios = config_project["run"]["name"]
print(f"Prefix: {prefix}")
print(f"Scenarios: {scenarios}")

Prefix: 2030-targets
Scenarios: ['targets+pci-pmi', 'targets+pci-pmi-national-h2-expansion', 'targets+greenfield-infra', 'targets+no-infra', 'only-emission-targets+no-infra']


In [5]:
# consolidate and rename
def rename_techs(label):
    prefix_to_remove = [
        "residential ",
        "services ",
        "urban ",
        "rural ",
        "central ",
        "decentral ",
    ]

    rename_if_contains = [
        "CHP",
        "gas boiler",
        "biogas",
        "solar thermal",
        "air heat pump",
        "ground heat pump",
        "resistive heater",
        "Fischer-Tropsch",
    ]

    rename_if_contains_dict = {
        "water tanks": "hot water storage",
        "retrofitting": "building retrofitting",
        # "H2 Electrolysis": "hydrogen storage",
        # "H2 Fuel Cell": "hydrogen storage",
        # "H2 pipeline": "hydrogen storage",
        "battery": "battery storage",
        "H2 for industry": "H2 for industry",
        "land transport fuel cell": "land transport fuel cell",
        "land transport oil": "land transport oil",
        "oil shipping": "shipping oil",
        # "CC": "CC"
    }

    rename = {
        "solar": "solar PV",
        "Sabatier": "methanation",
        "offwind": "offshore wind",
        "offwind-ac": "offshore wind (AC)",
        "offwind-dc": "offshore wind (DC)",
        "offwind-float": "offshore wind (Float)",
        "onwind": "onshore wind",
        "ror": "hydroelectricity",
        "hydro": "hydroelectricity",
        "PHS": "hydroelectricity",
        "NH3": "ammonia",
        "co2 Store": "DAC",
        "co2 stored": "CO2 sequestration",
        "AC": "transmission lines",
        "DC": "transmission lines",
        "B2B": "transmission lines",
    }

    for ptr in prefix_to_remove:
        if label[: len(ptr)] == ptr:
            label = label[len(ptr) :]

    for rif in rename_if_contains:
        if rif in label:
            label = rif

    for old, new in rename_if_contains_dict.items():
        if old in label:
            label = new

    for old, new in rename.items():
        if old == label:
            label = new
    return label


preferred_order = pd.Index(
    [
        "transmission lines",
        "hydroelectricity",
        "hydro reservoir",
        "run of river",
        "pumped hydro storage",
        "solid biomass",
        "biogas",
        "onshore wind",
        "offshore wind",
        "offshore wind (AC)",
        "offshore wind (DC)",
        "solar PV",
        "solar thermal",
        "solar rooftop",
        "solar",
        "building retrofitting",
        "ground heat pump",
        "air heat pump",
        "heat pump",
        "resistive heater",
        "power-to-heat",
        "gas-to-power/heat",
        "CHP",
        "OCGT",
        "gas boiler",
        "gas",
        "natural gas",
        "methanation",
        "ammonia",
        "hydrogen storage",
        "power-to-gas",
        "power-to-liquid",
        "battery storage",
        "hot water storage",
        "CO2 sequestration",
    ]
)

In [6]:
# iterate through all scenarios and store all pypsa networks in a dictionary with scenarios as keys
networks = {}
for scenario in scenarios:
    print("")
    print(f"Loading network for scenario {scenario}")
    n_path = project_dir + "/results/" + prefix+ "/"+scenario+"/postnetworks/base_s_90_lv1.05___2030.nc"
    networks[scenario] = pypsa.Network(n_path)


Loading network for scenario targets+pci-pmi


INFO:pypsa.io:Imported network base_s_90_lv1.05___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores



Loading network for scenario targets+pci-pmi-national-h2-expansion


INFO:pypsa.io:Imported network base_s_90_lv1.05___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores



Loading network for scenario targets+greenfield-infra


INFO:pypsa.io:Imported network base_s_90_lv1.05___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores



Loading network for scenario targets+no-infra


INFO:pypsa.io:Imported network base_s_90_lv1.05___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores



Loading network for scenario only-emission-targets+no-infra


INFO:pypsa.io:Imported network base_s_90_lv1.05___2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [18]:
# print all objectives
for scenario in scenarios:
    print("")
    print(f"Objective value in bn. €: {scenario}")
    print(round(networks[scenario].objective/1e9,3))


Objective value in bn. €: targets+pci-pmi
586.036

Objective value in bn. €: targets+pci-pmi-national-h2-expansion
585.817

Objective value in bn. €: targets+greenfield-infra
586.525

Objective value in bn. €: targets+no-infra
586.967

Objective value in bn. €: only-emission-targets+no-infra
580.175


In [19]:
import matplotlib.pyplot as plt

In [7]:
cost_df = networks[scenarios[0]].statistics.capex()

In [8]:
import matplotlib.pyplot as plt
import pypsa
import seaborn as sns
sns.set_theme("paper", style="whitegrid")


In [ ]:
n = pypsa.Network(snakemake.input.network)

n.loads.carrier = "load"
n.carriers.loc["load", ["nice_name", "color"]] = "Load", "darkred"
colors = n.carriers.set_index("nice_name").color.where(
    lambda s: s != "", "lightgrey"
)

def rename_index(ds):
    specific = ds.index.map(lambda x: f"{x[1]}\n({x[0]})")
    generic = ds.index.get_level_values("carrier")
    duplicated = generic.duplicated(keep=False)
    index = specific.where(duplicated, generic)
    return ds.set_axis(index)

def plot_static_per_carrier(ds, ax, drop_zero=True):
    if drop_zero:
        ds = ds[ds != 0]
    ds = ds.dropna()
    c = colors[ds.index.get_level_values("carrier")]
    ds = ds.pipe(rename_index)
    label = f"{ds.attrs['name']} [{ds.attrs['unit']}]"
    ds.plot.barh(color=c.values, xlabel=label, ax=ax)
    ax.grid(axis="y")
